coding=utf-8
Copyright 2024 The Google Research Authors.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

"""X-MAGICAL cross-embodiment pretraining script."""


In [1]:

import os.path as osp
import subprocess
import os
from configs.constants import XMAGICAL_EMBODIMENT_TO_ENV_NAME

from absl import app
from absl import logging
from configs.constants import ALGORITHMS
from configs.constants import EMBODIMENTS
from torchkit.experiment import string_from_kwargs
from torchkit.experiment import unique_id
import yaml
import random


2024-12-01 21:01:17.818567: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-01 21:01:17.818652: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-01 21:01:17.818676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 21:01:17.831111: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

# Mapping from pretraining algorithm to config file.
ALGO_TO_CONFIG = {
    "xirl": "configs/xmagical/pretraining/tcc.py",
    "lifs": "configs/xmagical/pretraining/lifs.py",
    "tcn": "configs/xmagical/pretraining/tcn.py",
    "goal_classifier": "configs/xmagical/pretraining/classifier.py",
    "raw_imagenet": "configs/xmagical/pretraining/imagenet.py",
}


In [3]:
# We want to pretrain on the entire 1k demonstrations.
MAX_DEMONSTRATIONS = -1
# embodiment = None
embodiment = "longstick"
algo = "xirl"
unique_name = True,
random_number = random.randint(1, 1000)  # You can adjust the range as needed
experiment_name = f"/home/user/xirl/exp/exp{random_number}"
experiment_name = "/home/user/xirl/exp/same_long_gnn"
dataset_folder = f'/tmp/xirl/datasets/xmagical/train/{embodiment}'
# device = 'cpu'
device = 'cuda:0'
embodiments = EMBODIMENTS if embodiment is None else [embodiment]
graph_data_path = f"./data/{embodiment}/graphs/combined_graph.pt"
print("experiment name: ",experiment_name)

experiment name:  /home/user/xirl/exp/same_long_gnn


In [4]:
for embodiment in embodiments:
    # Generate a unique experiment name.
    print("embodiment: ",embodiment)
    kwargs = {
        "dataset": "xmagical",
        "mode": "cross",
        "algo": algo,
        "embodiment": embodiment,
    }
    if unique_name:
      kwargs["uid"] = unique_id()
    logging.info("Experiment name: %s", experiment_name)
    
    # Train on all classes but the given embodiment.
    trainable_embs = tuple(EMBODIMENTS - set([embodiment]))

embodiment:  longstick


In [70]:
for embodiment in embodiments:
    # The 'goal_classifier' baseline does not need to compute a goal embedding.
      subprocess.run(
          [
              "python",
              "compute_goal_embedding_gnn.py",
              "--experiment_path",
              experiment_name,
              "--graph_data_path",
              graph_data_path,
              "--dataset_folder",
              dataset_folder
          ],
          check=True,
      )


/home/user/mambaforge/envs/env/lib/python3.11/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /home/user/mambaforge/envs/env/lib/python3.11/site-packages/libpyg.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/user/mambaforge/envs/env/lib/python3.11/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/user/mambaforge/envs/env/lib/python3.11/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/user/mambaforge/envs/env/lib/python3.11/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /home/user/mambaforge/envs/env/l

Loading chipmunk for Linux (64bit) [/home/user/mambaforge/envs/env/lib/python3.11/site-packages/pymunk/libchipmunk.so]


CalledProcessError: Command '['python', 'compute_goal_embedding_gnn.py', '--experiment_path', '/home/user/xirl/exp/same_long_gnn', '--graph_data_path', './data/longstick/graphs/combined_graph.pt', '--dataset_folder', '/tmp/xirl/datasets/xmagical/train/longstick']' returned non-zero exit status 1.

In [65]:
import utils

gnn_emb = utils.load_pickle(experiment_name, "gnn_emb.pkl")
gnn_distance_scale = utils.load_pickle(experiment_name, "gnn_distance_scale.pkl")

print(gnn_emb,gnn_distance_scale)


[[   2.4228632  174.6436       2.7645833  -68.840355    14.667976
   -15.482741  -185.1232      91.334465    28.79416   -120.26102
   -11.66102     71.56417    -61.6732      26.155985  -127.335724
    85.03038     51.98203    -32.865753    34.45231     53.460728
   -93.62486    -54.13439     -7.5887403   19.298368   135.26053
  -124.65146    117.01777   -226.27058    142.14024     61.62393
   136.7354    -170.62985  ]] 1.0
